In [ ]:
# Iterate over the different temperature settings
for temp in temperatures:

    # Repeat the whole experiment n times
    for experiment in range(NrOfExper):
        print(f"\n---\nStarting experiment {experiment+1}...\n---")
        training_data = []
        current_strength = 0.0
        iterations = 0
        formulations=[]
        # System message including both the role prompt and context
        system_message = system_role_prompt.value  + '\n' + context_prompt.value

        while iterations < budget:
            iterations += 1
            print(f"\n---\nStarting iteration {iterations} at temp {temp}...")

            # Start with the system message
            messages = [
                {"role": "system", "content": system_message}
            ]

            # Add the training data to the messages
            if training_data:
                messages.append({"role": "assistant", "content": "Previously, we have tested these formulations with the following result:\n" + "\n".join(training_data)})
                # Add the iteration prompt
                messages.append({"role": "user", "content": iterate_prompt.value})

            if conversation_dropdown.value == 'Complete':
                print("--- Conversation History ---")
                for msg in messages:
                    print(f"{msg['role']}: {msg['content']}")

            valid_solution = False
            while not valid_solution:
                if enable_recursive_strategy == 1:  # Assuming '1' means enabled
                    assistant_response = run_recursive_strategy(system_message, training_data,iterate_prompt.value,context_prompt.value, conversation_dropdown)
                else:
                    response = call_openai_api(messages, temp)
                    assistant_response = response['choices'][0]['message']

                if assistant_response['role'] == "assistant":
                    print('Model Response: ', assistant_response['content'])
                    suggested_solution = parse_solution(assistant_response['content'])

                    if suggested_solution is not None:
                        lab_result = find_matching_result(formulation_df, suggested_solution)
                        if lab_result:
                            current_strength = lab_result
                            training_data.append(f"{assistant_response['content']} resulted in a strength of {current_strength} MPa.")
                            valid_solution = True
                            print('The suggested formulation achieved a strength of ', current_strength, ' MPa.')
                        else:
                            print(f"Iteration {iterations}: No matching lab result found for suggestion {suggested_solution}")
                    else:
                        print(f"Iteration {iterations}: Assistant's response did not contain a valid solution. Trying again.")

                    if not valid_solution:
                        messages[-1]["content"] = iterate_prompt.value + "\nRemember the exact parameter grid: Powderkg: {360, 370, 380, 390,400, 410, 420, 430, 440, 450}, wc: {0.45, 0.5, 0.55, 0.6}, materials: {0.7/0.3, 0.6/0.4, 0.5/0.5}, curing: {ambient, heat}. Please stick to these values and the following format: 'The formulation is Powderkg = {your estimate}, wc = {your estimate}, materials = {your estimate}, curing = {your estimate}'"
                else:
                    print(f"Iteration {iterations}: Response not from 'assistant'. Trying again.")

            if current_strength >= desired_strength:
                print(f"\nDesired compressive strength of {desired_strength} MPa achieved after {iterations} iterations. The solution is {suggested_solution}.")
                break

        timestamp = str(int(time.time()))

        # create the file name
        filename = f"Results/ID/{model_dropdown.value}_{prompt_dropdown.value}_prompt_experiment_{experiment+1}_temp_{temp}_target_{int(targ_quant.value)}_%_Dev_Budget_{budget}_recursive_{enable_recursive_strategy}_{timestamp}.csv"
        # open the file in write mode
        with open(filename, 'w', newline='') as file:
            writer = csv.writer(file)

            # write the headers
            writer.writerow(["Formulation", "Compressive Strength"])

            # iterate over the training data
            for data in training_data:
                try:
                    # parse the data to extract formulation and compressive strength
                    formulation, strength_str = data.split(" resulted in a strength of ")
                    strength = float(strength_str.split(" ")[0])  # convert string to float
                    writer.writerow([formulation, strength])
                except ValueError as e:
                    print(f"Failed to parse data: {data}")
                    print(f"Error: {e}")

        print(f"Data for experiment {experiment+1} and temp {temp} successfully saved to {filename}.")

